- After Training the model We found out how much imbalanced Data Can effect the model, Which led us to implement the data augmentation concept to make all the number of images in all files the same 


## number of images we have before data augmentation

In [5]:
import os

def count_images_in_directory(directory_path):
    total_images = 0
    for root, dirs, files in os.walk(directory_path):
        image_files = [file for file in files if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
        total_images += len(image_files)
    return total_images

# Paths to your directories
# Set the directories to train and test and validate 
train_dir = r"C:\Users\khair\Downloads\archive (2)\train"
test_dir = r"C:\Users\khair\Downloads\archive (2)\test"
validation_dir = r"C:\Users\khair\Downloads\archive (2)\validation"

# Count images in each directory
train_count = count_images_in_directory(train_dir)
test_count = count_images_in_directory(test_dir)
validation_count = count_images_in_directory(validation_dir)

print(f"Number of images in train directory: {train_count}")
print(f"Number of images in test directory: {test_count}")
print(f"Number of images in validation directory: {validation_count}")


Number of images in train directory: 17427
Number of images in test directory: 2200
Number of images in validation directory: 2033


In [ ]:

import os

def count_images_in_class_folders(train_dir):
    class_counts = {}
    for class_name in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_name)
        if os.path.isdir(class_path):
            image_files = [file for file in os.listdir(class_path) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
            class_counts[class_name] = len(image_files)
    return class_counts


# Count images in each class within the train directory
train_counts = count_images_in_class_folders(train_dir)


print("Train Directory Image Counts by Class:")
for class_name, count in train_counts.items():
    print(f"{class_name}: {count}")


Train Directory Image Counts by Class:
actinolite: 24
Andalusite: 60
Argillaceous_siltstone: 880
biotite: 195
Bioturbated_siltstone: 1200
carbonate: 4000
coal: 3200
debris_rock: 702
granite: 60
hornblende: 72
Massive_calcareous_siltstone: 680
Massive_calcite-cemented_siltstone: 1160
olivine: 140
oolites: 82
plagioclase: 90
Porous_calcareous_siltstone: 640
pyroxene: 146
sandstone: 4000
staurolite: 96


In [8]:
# Count images in each class within the test directory
test_counts = count_images_in_class_folders(test_dir)

print("Test Directory Image Counts by Class:")
for class_name, count in test_counts.items():
    print(f"{class_name}: {count}")

Test Directory Image Counts by Class:
actinolite: 4
Andalusite: 14
Argillaceous_siltstone: 90
biotite: 25
Bioturbated_siltstone: 190
carbonate: 500
coal: 400
debris_rock: 86
granite: 16
hornblende: 17
Massive_calcareous_siltstone: 80
Massive_calcite-cemented_siltstone: 125
olivine: 16
oolites: 10
plagioclase: 9
Porous_calcareous_siltstone: 85
pyroxene: 20
sandstone: 500
staurolite: 13


In [9]:
# Count images in each class within the test directory
valid_counts = count_images_in_class_folders(validation_dir)

print("Test Directory Image Counts by Class:")
for class_name, count in valid_counts.items():
    print(f"{class_name}: {count}")

Test Directory Image Counts by Class:
actinolite: 6
Andalusite: 19
Argillaceous_siltstone: 55
biotite: 24
Bioturbated_siltstone: 110
carbonate: 500
coal: 400
debris_rock: 90
granite: 19
hornblende: 17
Massive_calcareous_siltstone: 70
Massive_calcite-cemented_siltstone: 120
olivine: 18
oolites: 11
plagioclase: 13
Porous_calcareous_siltstone: 30
pyroxene: 18
sandstone: 500
staurolite: 13
